In [ ]:
#Importing necessary libraries
import pandas as pd
import numpy as np
from google.colab import files

uploaded = files.upload()
df = pd.read_csv("reddit_data1.csv")
df.head()

Saving reddit_data1.csv to reddit_data1.csv


,Post
0,What age group are you in? I don't know how to...
1,Changes to r/RandomThoughts are coming Hey fri...
2,Orgasms seem kind of amazing. Not in the way t...
3,Isn't following an alpha male the literal defi...
4,Butt is weird Each of us has these two spheres...


In [ ]:
#create id column for sentiment scoring after
df["id"] = df.index
df.head()

,Post,id
0,What age group are you in? I don't know how to...,0
1,Changes to r/RandomThoughts are coming Hey fri...,1
2,Orgasms seem kind of amazing. Not in the way t...,2
3,Isn't following an alpha male the literal defi...,3
4,Butt is weird Each of us has these two spheres...,4


In [ ]:
df.shape

(2322, 2)

In [ ]:
#importing RoBERTa model for sentiment training
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import torch

tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

In [ ]:
#Creating function to assign sentiment score for a post
# 0 -> Negative 1-> Neutral 2 -> Positive
def sentiment(example):
  text = tokenizer(example, return_tensors='pt')
  result = model(**text)
  return int(torch.argmax(result.logits))

In [ ]:
#Assigning a sentiment score for each post in the dataframe
from tqdm.notebook import tqdm
res={}
for i, row in tqdm(df.iterrows(), total= len(df)):
  try:
    text = row['Post']
    id = row['id']

    res[id] = sentiment(text)
  except RuntimeError:
    print(f'Error at {i}')


  0%|          | 0/2322 [00:00<?, ?it/s]

Error at 436
Error at 734
Error at 930
Error at 933
Error at 1033
Error at 1070
Error at 1076
Error at 1082
Error at 1087
Error at 1213
Error at 1282
Error at 1306
Error at 1326
Error at 1343
Error at 1359
Error at 1362
Error at 1367
Error at 1368
Error at 1420
Error at 1505
Error at 1535
Error at 1537
Error at 1539


In [ ]:
#Building the new Dataframe
new_df = pd.DataFrame.from_dict(res, orient='index')
new_df = new_df.reset_index().rename(columns={'index': 'id', 0: 'Sentiment'})
new_df = new_df.merge(df, how='left')

meaning = {0: 'Negative', 1: 'Neutral',
            2: 'Positive'}
new_df['Meaning'] = new_df['Sentiment'].map(meaning)

In [ ]:
new_df.columns = ['id', 'Score', 'Post', 'Sentiment']
new_df = new_df.reindex(columns=['id', 'Post', 'Sentiment', 'Score'])
new_df.head()

,id,Post,Sentiment,Score
0,0,What age group are you in? I don't know how to...,Negative,0
1,1,Changes to r/RandomThoughts are coming Hey fri...,Neutral,1
2,2,Orgasms seem kind of amazing. Not in the way t...,Positive,2
3,3,Isn't following an alpha male the literal defi...,Negative,0
4,4,Butt is weird Each of us has these two spheres...,Neutral,1


In [ ]:
print(new_df['Sentiment'].value_counts())

Sentiment
Neutral     889
Positive    707
Negative    703
Name: count, dtype: int64


In [ ]:
new_df.shape

(2299, 4)

In [ ]:
#save dataframe
new_df.to_csv('sentiment_data1.csv', index=False)